# Image Processing - Segmentation

Jan Eglinger

![FMI](http://www.fmi.ch/img/logo-FMI-grey.gif)

<small>Facility for Advanced Imaging and Microscopy (FAIM)</small><br>
<small>Friedrich Miescher Institute for Biomedical Research (FMI)
Basel, Switzerland</small>

Basel, March 14, 2018

* Manual and semi-automatic tools

* Automated segmentation
  * Intensity thresholding
  * Textures and Structures (combining filtering with thresholding)

* Foreground/Background vs Instances: connected-component analysis
* Watershed and distance maps

* Other segmentation techniques
  * Clustering
  * Active contours
  * Region growing
  * Graph cut

* Morphological segmentation / gradient images
* Machine learning: trainable segmentation


In [14]:
//load ImageJ
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
%classpath add mvn net.imagej imagej 2.0.0-rc-71

//create ImageJ object
ij = new net.imagej.ImageJ()

notebook = ij.notebook()
datasetIO = ij.scifio().datasetIO()
ops = ij.op()
"ImageJ initialized"

ImageJ initialized

In [15]:
/* Required Imports */
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.interpolation.randomaccess.FloorInterpolatorFactory
import net.imglib2.RandomAccessibleInterval

/* Utility Functions */
tile = { images ->
  int[] gridLayout = images[0] in List ?
    [images[0].size, images.size] : // 2D images list
    [images.size] // 1D images list
  RandomAccessibleInterval[] rais = images.flatten()
  ij.notebook().mosaic(gridLayout, rais)
}

table_image = { array ->
    img = ij.op().create().kernel(array as double[][], new FloatType())
    ij.op().run("transform.scaleView", img,
        [32,32] as double[],
        new FloorInterpolatorFactory()
    )    
}

zoomedView = { img, factor ->
    ij.op().run("transform.scaleView", img,
        [factor,factor] as double[],
        new FloorInterpolatorFactory()
    ) 
}

label_display = { labels ->
    indexImage = labels.getIndexImg().copy()
    indexImage.firstElement().setReal(255.0) // hack to get full 0-255 display range
    
    view = ij.imageDisplay().createDataView(ij.dataset().create(indexImage))
    view.rebuild()
    lut = ij.lut().loadLUT(new File("luts/glasbey_inverted.lut"))
    view.setColorTable(lut, 0)
    view.rebuild()
    view
}
null

null

## Image Segmentation


* The process of partitioning a digital image into multiple segments

  * Foreground / Background

  * Multiple objects


## Image Segmentation

| Original Image                |
| ----------------------------- |
| ![](images/blob-original.png) |

* Image segments can be represented in different ways:

| Region of interest (ROI) | Mask | Label image |
|---|---|---|
| ![](images/blob-roi.png) | ![](images/blob-mask.png) | ![](images/blob-label.png) |


### Manual and semi-automatic segmentation

| Manual selection tools          |
| ------------------------------- |
| ![](images/selection-tools.png) |



| Magic wand                      |
| ------------------------------- |
| ![](images/magic-wand-tool.png) |


### Thresholding

* By defining a threshold on our intensity data, we can partition them into two groups, foreground and background

On an image, we can define our threshold on the **pixel intensities**:
* bright pixels will be considered foreground
* dark pixels will be considered background

For example, let's consider the (ideal) image of a spherical object:

In [3]:
disk = ops.run("create.img", [200, 200])
disk = ops.run("convert.uint8", disk)

// fill entire image with value 20
newValue = disk.firstElement().copy()
newValue.setReal(20)
disk = ops.run("image.fill", disk, newValue)

// fill disk with value 200
import net.imglib2.Point
import net.imglib2.algorithm.region.hypersphere.HyperSphere

location = new Point(disk.numDimensions())
location.setPosition([100, 100] as long[])

hyperSphere = new HyperSphere(disk, location, 50)

for (value in hyperSphere) {
  value.setReal(200)
}

disk = ops.run("filter.gauss", disk, 10)
ij.notebook().display([["Original": disk]])

Original


Now we can simply define a threshold somewhere between the brightest and the darkest intensity, which will divide our image into:
* foreground (white) and
* background (black):

In [4]:
thresholded = ops.run("threshold.otsu", disk)
ij.notebook().display([["Original": disk, "Thresholded": thresholded]])

Original Thresholded <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAAmmUlEQVR42u2dZ5c1VRGF74/AhFlERTFjAsSABLNiwoA5gwoKKmLAnDNmMaCoGAEToKKC6V+167lr7Vmbmjod7szwdt9bH+rLOx3OOdNP195Vp+ddHXfccV1FRUUeq1qEiooCpKKiAKmoKEAqKgqQiooCpKKiAKmoKEAqKgqQiooCpKKiogCpqChAKioKkIqKAqSiogCpqChAKioKkIqKAqSiogCpqChAaiEqKgqQiooCpKKiAKmoKEAqKgqQ3Ym73OUuhx61rgVIgVDgFCAFRAFTgBQQ67jrXe964ChgCpBFQ3EYENxZ8NTvtwA5cjDGPLh3u9vdDj0OE5j6fRcgdxoUfQ/13e9+9wPHptAULAXIkYIxBYi+B/we97jHxjEVnIPAUs9FAbIRFGOAyB7u448//tBiDDxjgSlQCpBDB2MIiPhA3/Oe99wX97rXvUZHdv4QNH3AFCgFyKGA0ZcpWkC0ILj3ve+dxn3uc59mtM5pwdMCpi+zFCgFyCAcY7NFHxAZDPGBv+9973uHuN/97jcY8ZwWRBk0GTBjs0pBsiOAHBSMLFNEKCIQGQD3v//99+IBD3jA5PDzM4BawERYssxSoOwoIIcBRh8UGRARghNOOGEvHvjAB+7FiSeeODr8PL9ehKcFzBAsBwGlANnirDEWjBYUnh0ciAjBgx70oHU8+MEP3ouHPOQho8PP07UiPBEYzzAtWKaCsmvZZFVZYxoYGRQRCAeBh/ukk05ax0Mf+tB1POxhD9uLk08+eTD8eF1D13R4BE0EJoNlKii7mk1WuwhHK2tEMNxXuHxyKGJ2EBACQQ/5wx/+8O4Rj3jEOh75yEfuxaMe9ajB8ON1Da7nAAkaAeNZJoPFQREsEZRNs0kBsiA4pmSNvoyRgdECQjA4AI9+9KO7xzzmMet47GMfu45TTjllLx73uMftC/+5ztE1uJ4D5NBkwIwBJWaUg2STAmRhcAyB4VkjAyPKpwwKASEYBAEP++Mf//h1POEJT+ie+MQnruNJT3rSXpx66qn7wn+uczhf1xJEAkfQMI4MlijD+kBxMz8GlG2FZLVLkmpITkUwooxStmhBEYEQBDzsp5122jpOP/307slPfvI6zjjjjHU85SlPaYaO0Tmcr2sJIu4TgWnBoqwS5VcfKH2ya9sl12qX4Mjk1JiM4WBIPkUolB0cCEHAg/7Upz61e9rTntY9/elPX8eZZ57ZPeMZz9iLs846a1/4zzle53IdrieAuI8DoywTYZEMa4EylFFiNtkFSFbbCkc04kNwuMdwMCSjPFsgZyIUyg4CQjDwYPOwn3322d0555zTnXvuuet45jOf2T3rWc/ai2c/+9n7wn/O8TqX63A9ritwuJ+AUZaJsDBuzyqSXw6Ke5QpkDgo2wTJapvh6JNULTmVZQweqJgtkDW8sQUFb/QIhIPwnOc8p3vuc5/bPe95z+ue//znr+MFL3jBOl74whc2Q8foHM7nOlzPwYnAMB7BwjglwzyrOCieUVqya4rk2hZIVrtgxoeyhlelWmCg7z1bIGuUKQQFb3YeWN7+PMCCgQf9vPPO6170ohd1L37xi7uXvOQl3Utf+tLuZS972TrOP//8ZugYjuc8zuc6XI/rChrux325P+MQLMoskmHKKsynBYpXvcZkk20274sHZBM4WlkDuYGZlcdAkkQwPFtEKHizCwjBIBBe/vKXd694xSu6V77yld2rXvWq7oILLljHq1/96nW85jWv2Rf6mY7lPM7nOlxP4AgaAcM4IiyeVSIokl7Mm/m77IrZ5DAgKUCOQfYYA4cklXuNLGu4x4hgKFsga5A4EQre8gKCh1kQ8MC/7nWv617/+td3b3jDG7o3vvGN63jTm960jje/+c37Qj/TsZzH+VxHAHF97iNguH+EhXEyXmWVCIp7lFY2kTeR5JoCyZKzyGrb4MjMuPsNSapW1pCcQrNLSgkM+QrezsgavIFDwZudtzwP7mtf+9o9GATAW9/61u5tb3tb9/a3v7278MIL13HRRRd173jHO5rBz3Us53E+1xFAgob7cV/uzzgcFsbJeBm3/IpAkfRivpJdfdnEIYm+pK/CtVRIVrsCh/sNhyNmDWSHzDfaXVLKwVC2QN4oUzgUPLhvectb1g8yD7UgeNe73tVdfPHF3SWXXNK9+93v7t7znves49JLL22GjuF4zuN8riN4uD734X7c12FRZmGcyioOiqQX85SZZ/4xm2SQuC/ZZkhW2yyrHA73Gy6psqyB/MDc0n9AwyNRBAZ637MFUoeHEgnEQ8obnre9gOChFgTvfe97u/e9733d+9///u7yyy/vPvCBD6zjiiuuaIaO4XjO43yuI3i4voDhvtyfcTAexsX4PKswfoHCvJgf82S+kl0xm7jkir6kBcm2yK3VrsAR/QYSQhUqeQ3eoi6nMLloeEkpgYHu5y2NvJF04uF85zvfuX7D87YXEDzUguBDH/pQ9+EPf7j7yEc+0l155ZXdRz/60XV87GMfa4aO4XjO43yuI3i4voDhvtyfcTAeSTHGyXgZt0CR9GJ+zNNlF+sgb6JKF+uV+ZJth2S15IrVJnDIb/BLd0nlWUNyCpNLOVVSKoLB2xqZwxtcUPCG520vIAQDD/vHP/7x7pOf/GT3qU99qvv0pz+9js985jPr+OxnP7sv9DMdy3mcz3W4nqARMNyX+wsWxsX4GGcERdKL+TFPyS7PJi65BAnrtykkS6xsrZZezh0Lh8y4/IZLKnkNzxqSU/QiJKUcDCQNb2tkjjKFoBAQn/jEJ9YPNA83D/znPve57vOf/3z3hS98ofviF7/YfelLX1rHl7/85WboGI7nPM7nOlyP63J97iNgBIsyC+NjnIzXQZH0Yn6SXZ5N5E1ccsmXyLxPgWSp5d/V0k35QeCQEUdauNdQ1kCOYHYpq2KAlTEcDN7aH/zgB9dvcqQQDytvet78PMiCgYf9K1/5SvfVr361+9rXvtZ9/etf76666qruG9/4xl5885vf3Av/d47jeM7jfK7D9QQN9+F+3Jf7Mw7Gw7gYn4OijMJ8mBfzY57KJu5NWBcZ+KOAZAlSa7Wk7NEHh/ocY+BwvyFJRScaTe5ZAzki842ml5SKYPD25k3uUPDW50HmoRYIPPjf+ta3um9/+9vdd77zne673/1u973vfe8O8f3vf3/fv3Ecx3Me5wsgrsv1uQ/3c1gYD+OKoEh6MR+Zeebp2YR1YD0kudyXjIWk1ScZgqQAOQTf4eXcrAlIlcU9R4RDvY0IB30DeQ3PGsgTqkWXXXbZHcDACyhbIH0iFDzIgkEP/9VXX9394Ac/6H74wx+u40c/+tE6fvzjH+8L/UzHch7nCyJBw30iLIxHWYVxOijMg/lIdimbyJuwDhkkrFsGiTxJ7JMIkqz8uxSptVqStMpMuW8f8T5HHxyYUHoA7jdcUqHRPWsgTzC/aHvPGPgAZQukj0PBW19ACAQe+muuuab7yU9+0v30pz/trr322nX87Gc/a4aO4XjO43zBw3UFDPdzWBiPsgrj9IzCPJgP8/JswrxdcrkvYb1Ytz5IYjNR21KGTPucpdZq6dKqBYdKuUNwyG/QVMO4Ijkoj8asgflF20tKKWMIDKSPMoWgEBCCgQf+5z//efeLX/yiu+6669bxy1/+ci9+9atf7YX/u47lPM4XOFxXwAgWZRbG46AoozB+5sF8YjZh3syfdWA95EuGIPEScAbJkqXWamnSqs+Uq0PufQ435MiECAc9AR4GdLgkFX0EtLpnDWQKVSM0fgYGb3EkEG92QeFACIBf//rX6/jNb37T/fa3v92L3/3ud/vCf87xOlcAOTCChfszDsaTgcL4mYdkl7IJ82XeklysB+vC+kRI5Elk3L1Poo77WNM+d6m1W

In reality, images are never free of **noise**.

Let's add two versions of our object, with increasing levels of noise:

In [5]:

noiseOp1 = ops.op("addNoise", disk.firstElement(), 0, 255,  40)
noiseOp2 = ops.op("addNoise", disk.firstElement(), 0, 255, 120)

noisy1 = ops.run("copy.img", disk)
ops.run("map", noisy1, noiseOp1)
noisy2 = ops.run("copy.img", disk)
ops.run("map", noisy2, noiseOp2)
ij.notebook().display([["Original": disk, "Noisy": noisy1, "Very Noisy": noisy2]])



Original Noisy Very Noisy <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAAmmUlEQVR42u2dZ5c1VRGF74/AhFlERTFjAsSABLNiwoA5gwoKKmLAnDNmMaCoGAEToKKC6V+167lr7Vmbmjod7szwdt9bH+rLOx3OOdNP195Vp+ddHXfccV1FRUUeq1qEiooCpKKiAKmoKEAqKgqQiooCpKKiAKmoKEAqKgqQiooCpKKiogCpqChAKioKkIqKAqSiogCpqChAKioKkIqKAqSiogCpqChAaiEqKgqQiooCpKKiAKmoKEAqKgqQ3Ym73OUuhx61rgVIgVDgFCAFRAFTgBQQ67jrXe964ChgCpBFQ3EYENxZ8NTvtwA5cjDGPLh3u9vdDj0OE5j6fRcgdxoUfQ/13e9+9wPHptAULAXIkYIxBYi+B/we97jHxjEVnIPAUs9FAbIRFGOAyB7u448//tBiDDxjgSlQCpBDB2MIiPhA3/Oe99wX97rXvUZHdv4QNH3AFCgFyKGA0ZcpWkC0ILj3ve+dxn3uc59mtM5pwdMCpi+zFCgFyCAcY7NFHxAZDPGBv+9973uHuN/97jcY8ZwWRBk0GTBjs0pBsiOAHBSMLFNEKCIQGQD3v//99+IBD3jA5PDzM4BawERYssxSoOwoIIcBRh8UGRARghNOOGEvHvjAB+7FiSeeODr8PL9ehKcFzBAsBwGlANnirDEWjBYUnh0ciAjBgx70oHU8+MEP3ouHPOQho8PP07UiPBEYzzAtWKaCsmvZZFVZYxoYGRQRCAeBh/ukk05ax0Mf+tB1POxhD9uLk08+eTD8eF1D13R4BE0EJoNlKii7mk1WuwhHK2tEMNxXuHxyKGJ2EBACQQ/5wx/+8O4Rj3jEOh75yEfuxaMe9ajB8ON1Da7nAAkaAeNZJoPFQREsEZRNs0kBsiA4pmSNvoyRgdECQjA4AI9+9KO7xzzmMet47GMfu45TTjllLx73uMftC/+5ztE1uJ4D5NBkwIwBJWaUg2STAmRhcAyB4VkjAyPKpwwKASEYBAEP++Mf//h1POEJT+ie+MQnruNJT3rSXpx66qn7wn+uczhf1xJEAkfQMI4MlijD+kBxMz8GlG2FZLVLkmpITkUwooxStmhBEYEQBDzsp5122jpOP/307slPfvI6zjjjjHU85SlPaYaO0Tmcr2sJIu4TgWnBoqwS5VcfKH2ya9sl12qX4Mjk1JiM4WBIPkUolB0cCEHAg/7Upz61e9rTntY9/elPX8eZZ57ZPeMZz9iLs846a1/4zzle53IdrieAuI8DoywTYZEMa4EylFFiNtkFSFbbCkc04kNwuMdwMCSjPFsgZyIUyg4CQjDwYPOwn3322d0555zTnXvuuet45jOf2T3rWc/ai2c/+9n7wn/O8TqX63A9ritwuJ+AUZaJsDBuzyqSXw6Ke5QpkDgo2wTJapvh6JNULTmVZQweqJgtkDW8sQUFb/QIhIPwnOc8p3vuc5/bPe95z+ue//znr+MFL3jBOl74whc2Q8foHM7nOlzPwYnAMB7BwjglwzyrOCieUVqya4rk2hZIVrtgxoeyhlelWmCg7z1bIGuUKQQFb3YeWN7+PMCCgQf9vPPO6170ohd1L37xi7uXvOQl3Utf+tLuZS972TrOP//8ZugYjuc8zuc6XI/rChrux325P+MQLMoskmHKKsynBYpXvcZkk20274sHZBM4WlkDuYGZlcdAkkQwPFtEKHizCwjBIBBe/vKXd694xSu6V77yld2rXvWq7oILLljHq1/96nW85jWv2Rf6mY7lPM7nOlxP4AgaAcM4IiyeVSIokl7Mm/m77IrZ5DAgKUCOQfYYA4cklXuNLGu4x4hgKFsga5A4EQre8gKCh1kQ8MC/7nWv617/+td3b3jDG7o3vvGN63jTm960jje/+c37Qj/TsZzH+VxHAHF97iNguH+EhXEyXmWVCIp7lFY2kTeR5JoCyZKzyGrb4MjMuPsNSapW1pCcQrNLSgkM+QrezsgavIFDwZudtzwP7mtf+9o9GATAW9/61u5tb3tb9/a3v7278MIL13HRRRd173jHO5rBz3Us53E+1xFAgob7cV/uzzgcFsbJeBm3/IpAkfRivpJdfdnEIYm+pK/CtVRIVrsCh/sNhyNmDWSHzDfaXVLKwVC2QN4oUzgUPLhvectb1g8yD7UgeNe73tVdfPHF3SWXXNK9+93v7t7znves49JLL22GjuF4zuN8riN4uD734X7c12FRZmGcyioOiqQX85SZZ/4xm2SQuC/ZZkhW2yyrHA73Gy6psqyB/MDc0n9AwyNRBAZ637MFUoeHEgnEQ8obnre9gOChFgTvfe97u/e9733d+9///u7yyy/vPvCBD6zjiiuuaIaO4XjO43yuI3i4voDhvtyfcTAexsX4PKswfoHCvJgf82S+kl0xm7jkir6kBcm2yK3VrsAR/QYSQhUqeQ3eoi6nMLloeEkpgYHu5y2NvJF04uF85zvfuX7D87YXEDzUguBDH/pQ9+EPf7j7yEc+0l155ZXdRz/60XV87GMfa4aO4XjO43yuI3i4voDhvtyfcTAeSTHGyXgZt0CR9GJ+zNNlF+sgb6JKF+uV+ZJth2S15IrVJnDIb/BLd0nlWUNyCpNLOVVSKoLB2xqZwxtcUPCG520vIAQDD/vHP/7x7pOf/GT3qU99qvv0pz+9js985jPr+OxnP7sv9DMdy3mcz3W4nqARMNyX+wsWxsX4GGcERdKL+TFPyS7PJi65BAnrtykkS6xsrZZezh0Lh8y4/IZLKnkNzxqSU/QiJKUcDCQNb2tkjjKFoBAQn/jEJ9YPNA83D/znPve57vOf/3z3hS98ofviF7/YfelLX1rHl7/85WboGI7nPM7nOlyP63J97iNgBIsyC+NjnIzXQZH0Yn6SXZ5N5E1ccsmXyLxPgWSp5d/V0k35QeCQEUdauNdQ1kCOYHYpq2KAlTEcDN7aH/zgB9dvcqQQDytvet78PMiCgYf9K1/5SvfVr361+9rXvtZ9/etf76666qruG9/4xl5885vf3Av/d47jeM7jfK7D9QQN9+F+3Jf7Mw7Gw7gYn4OijMJ8mBfzY57KJu5NWBcZ+KOAZAlSa7Wk7NEHh/ocY+BwvyFJRScaTe5ZAzki842ml5SKYPD25k3uUPDW50HmoRYIPPjf+ta3um9/+9vdd77zne673/1u973vfe8O8f3vf3/fv3Ecx3Me5wsgrsv1uQ/3c1gYD+OKoEh6MR+Zeebp2YR1YD0kudyXjIWk1ScZgqQAOQTf4eXcrAlIlcU9R4RDvY0IB30DeQ3PGsgTqkWXXXbZHcDACyhbIH0iFDzIgkEP/9VXX9394Ac/6H74wx+u40c/+tE6fvzjH+8L/UzHch7nCyJBw30iLIxHWYVxOijMg/lIdimbyJuwDhkkrFsGiTxJ7JMIkqz8uxSptVqStMpMuW8f8T5HHxyYUHoA7jdcUqHRPWsgTzC/aHvPGPgAZQukj0PBW19ACAQe+muuuab7yU9+0v30pz/trr322nX87Gc/a4aO4XjO43zBw3UFDPdzWBiPsgrj9IzCPJgP8/JswrxdcrkvYb1Ytz5IYjNR21KGTPucpdZq6dKqBYdKuUNwyG/QVMO4Ijkoj8asgflF20tKKWMIDKSPMoWgEBCCgQf+5z//efeLX/yiu+6669bxy1/+ci9+9atf7YX/u47lPM4XOFxXwAgWZRbG46AoozB+5sF8YjZh3syfdWA95EuGIPEScAbJkqXWamnSqs+Uq0PufQ435MiECAc9AR4GdLgkFX0EtLpnDWQKVSM0fgYGb3EkEG92QeFACIBf//rX6/jNb37T/fa3v92L3/3ud/vCf87xOlcAOTCChfszDsaTgcL4mYdkl7IJ82XeklysB+vC+kRI5Elk3L1Poo77WNM+

Now, we'll try to get a segmentation by creating a binary image using *Otsu* threshold on the pixel intensities:

In [6]:
thr_noisy1 = ops.run("threshold.otsu", noisy1)
thr_noisy2 = ops.run("threshold.otsu", noisy2)

ij.notebook().display([["Thresholded": thresholded, "Thresholded Noisy": thr_noisy1, "Thresholded Very Noisy": thr_noisy2]])


Thresholded Thresholded Noisy Thresholded Very Noisy <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAACZklEQVR42u3TwQkAIAwEweu/aa3AlwgxmYNtIGGSZEk65ggSIBIgEiASIBIgEiASIBIgEiCSAJEAkQCRAJEAkQCRAJEAkQCRAJEEiASIBIgEiASIBIgEiASIBIgkQCRAJEAkQCRAJEAkQCRAJEAkQCQBIgEiASIBIgEiASIBIgEiASIJEAmQct3M/QCBARpAoIAFEChgAQSKH+ZfgIABCiBggAIIHJAAAgYogMABCSBggAIIHJAAAodBAggckAACBySAwAEJIHBAAggckAACBySAwAEJIIAAAggcBgkcBgkgBgggcEACCByQAAIIIIDAAQkggAACCBwGCCAGCCAGSG8gBgkggAACCCCAAAIHJIAAAggggBggcNgoJIAYIIAYIIAYIIAAAgggcEACCCCAAAIIIIAAYoAAYoAAYoAAYoAAYoAAAggggAACCCCAAAIIIIAAAogBAogBAogBAonBAYgBAogBAggggAACCCCAQAIHIIAAAgggBggkNgkHIAYIIAYIIJDAAQgggAACCCCAQAIHIIAAAggkcAACiAECicExDwgkcAACCCCAQAIHIJDAAQgggAACCRyAQAIHIJDAAQgkBgcgkMABCCRwAAIJHIBAAgcgkMABCChgAAIJHICAAgYgaozEPwEBBQxAQAEDEFigAAQWKACBBQpA9A6N+wEiASIBIgEiASIBIgkQCRAJEAkQCRAJEAkQCRAJEEkOIAEiASIBIgEiASIBIgEiASIBIgkQCRAJEAkQCRAJEAkQCRAJEAkQR5AAkQCRAJEAkQCRAJEAkQCRAJEEiASIBIgEiASIVKwNwcmy+f6oIzgAAAAASUVORK5CYII=" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAAMZElEQVR42u1d244dOQic///prPYhUnaVNgUGzKVa8kMyM+e0bcpcCvDPz8/Pr9/j3+fPf3cYHd9ZM4ffP5swz6Z7PGPxJglQ1lw8vmcBcPed4l/vW3Ee0e/UZe9u3/Pi7+MWa7NZwEENQv+DhwMBMt3Z2yTI3eca9f7C5/KU4KgNpoh3UXwmN4mjvjNt+Vwn+dklqJXDwtXeaVIIHeGawjUIspCei529cdnz46CT/gQk0t9QyN+vhee+Xs6h1yZlbBgBQpCmAYTCxrHOxPISeoKHYxxAukR/fj8Eb9wBtmSNyGdEvfuto0mQltjzfSnaHQHONJFHozMvwcHIVYqJFU3jd3bqp4Ke/hmzeUdzNVvAX+AdKdxTozvMNyNAWHBFsyh6jnNPnhd+0uSafRKFHBwc/z8ouBA8reeTlC5dTSiMfDeO4hWFFBSOYvJBIe6weVzzZ2tVayOmlLVqsokrtwAlMAefDJMTKKlN0w5TbhLnxjVJAQhDg2yyMLDEgCcPa0H6rn3C2jDzk6PeepaqB2FG6a65Pe516/IOiWt//6Wso6ZmvC0BKGzK0pzJAPnD5stp33HR/5Y8SPaG8uqyPhqu+FrSlOBgkZgZIFMuUqnYCZ6HQ80CtxANEpFPVIH0yrojI/p6OAZSCptYneLpkc50cg4RNVhHorBr+ka1jNspKSWtKgp5irzjAapoRsrAIRdrYmrBn6d/hVqX//sCFMgGJhbt3XeOdVRhldbPqUpklg/zcuiEWyt0ERqOWsl17bgoGYmAiKbQ/PwlZ8Dm1ckcQDUV62VuSNpB0kBfn/H1mdYwMjUOAJBMYZoaOjxpCSSR78/nb///4uAieJj2bT51EQH+Evavv0E+++uzvn4W6d8s0ERMcPMwqywRrS/hPmmn0/dEp5vfRO2ifaZAcNLOvBH8k3BrtMzX72q4E9Qkq8bFMN29eaaodGp/Cd7p/xDT6PT7kgl3Aq8lovYyaZMAaaRFvoTt9Hyd4if/RBL2kxaw+B8ocDqWHrCi0OHzrPa7Rhuc/A1UG0if+/VeWs6GFYfFAVKhozxq70vmFaoZJKBIAEB8nqmcV6C8McVZMjckQT8JKmqWoX4KAiyUvIyOZg0B29wT4NZORmx+lIOw+CoWE03j/5y034TiNwIkuY5C40CjYPAAEerv3BKKS82vnidB5tUGGsHTCHvUg7LuKDci5YBNCPNe+SATNATiRGsFS3vqe3+GF5i+1qFaQmhGxFIEyAQ1qnUiEYLPQuC91iqadJaXWqN4/wMmKZ7SyyXAoGHYCEG/0RZWnySCOyl+xQQZczQF/VZoK/gnWg6GznoSA1o5PFhViG+iYLcA9k6Zb3xH/a4TAc2N0gpfZWChfAlS6di1E4tLX6yNTRasOU+VQrpR79LJShjBg2S3AZVyrLqZVZ4+h9UEu9EiFVuuUoMAyYIdHO5IDYGakNbamerdVYQ5zI5IoVmt0eZKJ+2CFHtZ/ZJiHAfyOb2SCL2dcbT8taugV2HcM3t5OcvXvujVbcIgn18mpj3zYCRAhJCkBAICQh8Ojm74UDQKtqOk9jU4OhKQaDRseKHdvLoNKYN3q/B7Mfaob5dZ984w72UUy4sL6O6AWz8HddJfhX0DAdencOlGu2jYcfonNu2i8UuscpNBNo/TIBr+g8+d9rD4IM19lXkhXK0pReD4pK28skSCAdY/eiVpkInEXQVnXUsCMxfrQX35DRnYXYBfgx/Jkh5AG+xo50MA+HwfGvYdme7eaXInLVJZS0x4H4T/sDTErkYsj2raoI1YRaWPb3PQNQKaIMzxPkgXLYI6g4xY5aScnHgRVhQ+ytc/qfwsZnmTT4P2AotKZnx0kM+sN0fDlHzumXOp0UMFEFww8P18jNO/qwNjIjilVKDmZta89qJRQknNg99wNag2pBZXcZOoKJlZGaHPLX4Hqk0GXM40s3uJ1cHkY2PTkSYPTbvCz3HOUcEnKGK10+vIqPM79L9q62tz+LzhRYZUEv4XIJYX91SdloiWtAl0umPnLWlp796+j8DVt+bco4KQTwwvEn2nSGK/rbr3cmhrzjWg2Awej7lbL9tpa2JNjWBtAkJ0SNfCgSAahgBJSG9HWtLweWtuNWXU415cc3JYgwSasG7HisIJAH9FHjp9DjsnEhRx5KG2ZewrWTv8bk9w/E2df51YXjY2H5220PAjhVNS6laA3Ua0+Lxj0k+k7djWoxFUvxex+Dc1T+2Q55R7+B5FrZXeRVEog85kxTh/SMqJax/F6toXyyO8S4DkhHytDa4LaJSWjpOpaIpgiGHPT8GRAfeH9O7F68WkEzz2tSjeW3efiXXjwG8Ehfc8PW+haiBLPcwo6aS68UOoSfROOdpA7mR60cRKQDmyMWTBcyJYJyCtjmJlTvxEEDKUW4MkfH2rVAmAVD0VJBNgo2mU1eHldSiXUawFVx5MCPd2IQnBQ/69qrP6HhomnaZW7HpUjFyl1YNUmrBmA24jWlsdeW1tzSl7ekCHxXnNq0/hSGoRf5B4gqKg9mlZ5WVy0pHTkI9tnZA0k6bm108LZ8pCRpHLyAFNxqH4UC5r98BC/hZhaysBoQqBeds2KaMQij5IQAYvktrAx5aUeNOPt5N59ce7jrqy9zOacnOqbgnVotwG6oOg4CkOnFnAkMgq+hwx5plnD972ADlN/NYW1XIyUtjRI32CxGBOuLcooHq8uJVt9yiYmqZhPLu7D+i927sexEoU3tw6Rccc60Zpbc/UTM5mlUgiJtcGgtA6N6vGzSaBRzvpGROV2F1qFFt4F71ReJC51afNj9V51wrA1JCwhzbybik6zsSqeDogkZNtjriXxkD9kMGjb9cSTVgY5UQ6ACmjQlBTa44Uq61y0qt1eY/M8p2QmuJ5oWlkYRQKvnU+SMTCIC

The result isn't very satisfying. We need to do some **preprocessing** before segmentation.

### Combining Filtering with Thresholding


Let's filter our noisy images with a Gaussian kernel and then binarize it with an *Otsu* threshold:


In [7]:
noisy1_blurred = ops.run("filter.gauss", noisy1, 5)
noisy2_blurred = ops.run("filter.gauss", noisy2, 20)

ij.notebook().display([["Noisy blurred": noisy1_blurred, "Very noisy blurred": noisy2_blurred]])

Noisy blurred,Very noisy blurred


... and then binarize it with an *Otsu* threshold:


In [8]:
noisy1_blurred_thr = ops.run("threshold.otsu", noisy1_blurred)
noisy2_blurred_thr = ops.run("threshold.otsu", noisy2_blurred)

ij.notebook().display([["Noisy blurred + thresholded": noisy1_blurred_thr, "Very noisy blurred + thresholded": noisy2_blurred_thr]])

Noisy blurred + thresholded,Very noisy blurred + thresholded
"<img src=""data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAACaUlEQVR42u3csREAIRADMfffNDQAGQFgeWYbOF7pJ8mQtM0RJEAkQCRAJEAkQCRAJEAkQCRAJAEiASIBIgEiASIBIgEiASIBIgEiCRAJEAkQCRAJEAkQCRAJEAkQSYBIgEiASIBIgEiASIBIgEiASIBIAkQCRAJEAkQCRAJEAkQCRAJEEiASIBIgL7eauwBS+eGfnBsDAgQ0gIABCyBggAIIHKAAAsfD836AgAEKIHCAAggckAACBiSAwAEKIHBAAggYkAACh0ECCByQAAIHJIDAAQkgcEACCByQAAIHJIDAAQkggAACCBzWiwQQA6QaiEECCCCAAAIHJIAAAgggcAACCCCQAAKHAQKIAQKHQQKIAQKIAQIIIJAAAgcggAACCCCAAAIIIIAAAojVIAHEAKkAYoAAAggggAACCSCAAAIIIIAAAogBAogBAogBAogBAogBAggggAACCSCAAAIIIIAAAgggcAACiAECiAHiv1gGCCAGByCAAAIIIIAAAggcgEACCCCAAAIIIAYIJNaFAxADBBADBBBI4AAEEEAAAQQOQCABBBBA4AAEEjgAgcQAAcTg6AUCCRyAQAIHIJDAAQgkcAACCRyAQAIHIJDAAQgkcAACCRwCBBQ4AIEEDkBAgQMQUMAABBQwAAEFDEBAAQMQ3YzFewACChiAAAMFINDAAIgEiASIBIgkR5AAkQCRAJEAkQCRAJEAkQCRAJEEiASIBIgEiASIBIgEiASIBIgEiCRAJEAkQCRAJEAkQCRAJEAkQCQBIgEiASIBIgEiASIBIgEiASIBIgkQCRAJEAkQCRAJEAkQ6bMmo5lVH49OhqQAAAAASUVORK5CYII="" />","<img src=""data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAYAAACtWK6eAAACb0lEQVR42u3dsQ0AIQwEQfffNFRAhpDxzUnbAPyEL6qqlqRjDkECRAJEAkQCRAJEAkQCRAJEAkQSIBIgEiASIBIgEiASIBIgEiASIJIAkQCRAJEAkQCRAJEAkQCRAJEEiASIBIgEiASIBIgEiASIBIgEiCRAJEAkQCRAJEAkQCRAJEAkQCQB8q4bc46AwAAMIFAAAwgUsAACRve5L0DAAAUQMEABBA5QAAEDFEDgAAUQMCABBA5IAIHDQAEEDkgAgQMSQOCABBA4IAEEDkgAgQMSQACBBBA4LAwJHAZILBCDBBBAIAEEDkgAgQMQQACBBBA4AAEEEBuPBA4DBBCDJB2IAQIIIJAAAggggMABCCCAQAIIIAYIHAYIIAYJIAYIIAYIIIAAAggckAACCCCAAAIIIIAYIIAYIIAYIIAYIIAYJIAYIIAAAggggAACCCCAAAIIIIAAYoAAYoAAYoAAYoAAYoD45dYXCAgggMABCCCAAAIIIIAAYoBAYoAAYoAAYnB4gs0AAcQAAQQQQACBBA5AAAEEEEgAAQQQm4YDEAMkDggkcAACCSCAAAIHIJAAAgggcAACCSCAQGIxOHKAQAIHIIDAAQgkgAACCRyAQAIHIJDAAQgkcAACCRyAgGJwAAIJGICAAgcgkIABCChwAAIKGICAAgYgoIABiH7C4o4AgQUMQIABAhBoYABEAkQCRAJEAkQSIBIgEiASIBIgEiASIBIgEiCSHIIEiASIBIgEiASIBIgEiASIBIgkQCRAJEAkQCRAJEAkQCRAJEAkQCQBIgEiASIBIgEiASIBIgEiASIJEAkQCRAJEAkQqVUbvr080gUfmrEAAAAASUVORK5CYII="" />"


Unfortunately, it isn't always as easy:

In [9]:
image = datasetIO.open("https://2.bp.blogspot.com/-uuoxxToW-o4/WRRaSRBRx2I/AAAAAAAAZVg/jV5tmaapBNUGUXvbv4p6h41JWuvYCakeQCEw/s1600/levedura.png")
image = ops.run("convert.float32", ops.run("hyperSliceView", image, 2, 0))

But also here, image filtering helps. In this case, we use a *Variance* filter:

In [10]:
filtered = ops.run("copy.img", image)
import net.imglib2.algorithm.neighborhood.HyperSphereShape
filtered = ops.run("filter.variance", filtered, image, new HyperSphereShape(15))
ij.notebook().display([["Variance-filtered": filtered]])

Variance-filtered
""


In [11]:
thresholded = ops.run("threshold.otsu", filtered)
ij.notebook().display([["Thresholded": thresholded]])

Thresholded


* Other option:
  * **Local** thresholding within a given pixel neighborhood
    * Open the *Dot Blot* sample image in ImageJ (*File › Open Samples › Dot Blot (7K)*)
    * Run *Image › Adjust › Auto Local Threshold* (with the *Try all* option)

## Connected-component analysis

Differentiating between foreground and background often is not enough. In most cases, we'd like to get an idea about the **objects** in our image.

* How many distinct objects (e.g. cells) are there?
* Can we get measurements (e.g. size, intensity) for each object

So we need to uniquely label all pixels that belong to a *single* connected object. We apply a **connected-component analysis**.

Each connected component of our image gets assigned a unique label (i.e. pixel intensity):

In [12]:
import net.imglib2.algorithm.labeling.ConnectedComponents.StructuringElement

labels = ops.run("cca", thresholded, StructuringElement.FOUR_CONNECTED)

ij.notebook().display([["Label map": labels.getIndexImg(), "with false-color LUT": label_display(labels)]])

Label map,with false-color LUT


### Watershed

When the number of objects is large, or they're highly clustered, a connected-component analysis will not yield the desired object labeling. Touching objects will be assigned the same label and therefore considered a single object.

A **watershed segmentation** helps to distinguish touching objects.

We'll have a look interactively with ImageJ's watershed implementation.


In [13]:
image = datasetIO.open("https://imagej.net/_images/8/83/NucleiDAPIconfocal.png")
image = ops.run("convert.float32", image)

In [ ]:
blurred = ops.run("gauss", image, 4)
thresholded = ops.run("threshold.otsu", blurred)

### Distance maps

In order to separate particles exclusively by their morphology, we can look at their local thickness using the **distance transform**, also called **distance map**


In [ ]:
transformed = ops.run("distancetransform", thresholded)

In [ ]:
inverted = ops.run("create.img", transformed)
ops.run("image.invert", inverted, transformed)

In [ ]:
//blurred_distMap = ops.run("gauss", inverted, 3)
result = ops.run("watershed", blurred_distMap, true, false)
label_display(ops.image().watershed(result, blurred_distMap, true, false, 3, thresholded))

### MorphoLibJ

* [MorphoLibJ](https://imagej.net/MorphoLibJ) is an ImageJ plugin offering powerful segmentation implementations


## Other segmentation methods

* Clustering (e.g. [k-means clustering plugin](http://ij-plugins.sourceforge.net/plugins/segmentation/k-means.html))
* Region growing / Active contours (e.g. [Level Sets](https://imagej.net/Level_Sets))
* Graph cut (e.g. [Graph Cut](https://imagej.net/Graph_Cut))


## Machine learning: trainable segmentation

* Machine learning segmentation makes use of filtering the image (with various filter sizes) to create various "features" on which a classifier can be trained.

  * [Trainable Weka Segmentation](https://imagej.net/Trainable_Weka_Segmentation) in ImageJ
  * ilastik ([ilastik.org](https://www.ilastik.org/))

## Resources

* https://imagej.net/Segmentation
* https://forum.image.sc
